In [66]:

import os
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
import time
from dotenv import load_dotenv
import bs4
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFLoader

load_dotenv()
from langchain_community.document_loaders import PyPDFDirectoryLoader
## load the Groq API key
groq_api_key = os.environ['GROQ_API_KEY']

In [67]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

c:\Users\Ritesh Kapoor\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [68]:
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader


In [69]:
embeddings=hf
url = "https://www.datacamp.com/cheat-sheet/getting-started-with-python-cheat-sheet"
loader = RecursiveUrlLoader(
    url=url, max_depth=20, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

print(docs)
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=250) 
final_documents=text_splitter.split_documents(docs) 
vectors=FAISS.from_documents(final_documents,embeddings) 


[Document(page_content='Just a moment...Enable JavaScript and cookies to continue', metadata={'source': 'https://www.datacamp.com/cheat-sheet/getting-started-with-python-cheat-sheet', 'content_type': 'text/html; charset=UTF-8', 'title': 'Just a moment...', 'language': 'en-US'})]


In [70]:

llm=ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")


prompt=ChatPromptTemplate.from_template(
"""
you are a coding test platform.
I have give you a context of python documentation. 
You need to ask questions only on python language  <context>
{context}
<context>
Questions:{input}

"""
)
document_chain = create_stuff_documents_chain(llm, prompt)
retriever = vectors.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)



In [82]:

response=retrieval_chain.invoke({"input":'create 5 random challenging objective type question on python language generate only 3 options out of which only 1 is correct. tell correct option     '})

print(response['answer'])

Question 1: What is the output of the following code snippet?

```python
class A:
    def __init__(self, x):
        self.x = x

a = A(5)
print(a.x + 2)

Options:
A) 7
B) 5
C) 2

Correct Option: A
```

Question 2: What is the output of the following code snippet?

```python
def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)

print(fibonacci(10))

Options:
A) 55
B) 34
C) 89

Correct Option: A
```

Question 3: What is the output of the following code snippet?

```python
numbers = [1, 2, 3, 4, 5]

print(list(filter(lambda x: x % 2 == 0, numbers)))

Options:
A) [1, 3, 5]
B) [2, 4]
C) [1, 2, 3, 4, 5]

Correct Option: B
```

Question 4: What is the output of the following code snippet?

```python
class Person:
    def __init__(self, name, age):
        self.name = name
        self.__age = age

    def get_age(self):
        return self.__age

p = Person("John", 25)
print(p.get_age())

Options:
A) 25
B) John
C) Error

Correct Option: A


In [78]:
from llama_index.llms.groq import Groq
from dotenv import load_dotenv
import os 
load_dotenv()

## load the Groq API key
groq_api_key = os.environ['GROQ_API_KEY']
llm = Groq(model="llama3-70b-8192", api_key=groq_api_key)

In [81]:
from llama_index.core.llms import ChatMessage
answer=response
messages = [
    ChatMessage(
        role="system", content="you are an intelligent coding test checker "
    ),
    ChatMessage(role="user", content=f"{answer} check the answers and if the answer is wrong then add none of the above as d) and give it a answer"),
]
resp = llm.chat(messages)
print(resp)

assistant: Here are the 5 challenging objective type questions on Python language with 3 options each, out of which 1 is correct:

**Question 1:**
What is the output of the following code?
```
def fibonacci(n):
    if n <= 0:
        return "Invalid input"
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        a, b = 0, 1
        for _ in range(n - 2):
            a, b = b, a + b
        return b
print(fibonacci(7))
```
A) 5
B) 8
C) 13

**Answer:** C) 13

**Question 2:**
Which of the following is the correct way to define a default argument with a mutable object in Python?
```
# Option 1
def add_item(items=[]):
    items.append('new_item')
    return items

# Option 2
def add_item(items=None):
    if items is None:
        items = []
    items.append('new_item')
    return items

# Option 3
def add_item(items=list()):
    items.append('new_item')
    return items
```
A) Option 1
B) Option 2
C) Option 3

**Answer:** B) Option 2

**Question 3:**
What is the